# Voxelize fibril point clouds 

## Imports & define paths & quick functions

In [ ]:
# Imports
import pathlib
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
from scipy.spatial.transform import Rotation as R
from tqdm import tqdm

from custom_fibril_gen.custom_fibril_gen_parallel_v1 import *

In [ ]:
# Define paths
notebookPath = pathlib.Path.cwd()
# RBDsPath = notebookPath.joinpath('blender_RBD_positions/RBD01_2.13umXY-0.1umZ-film_20nmXZ-100nmY-RBDs_300frames_0.1grav')
# RBDsPath = notebookPath.joinpath('blender_RBD_positions/RBD02_40x200nmFibrils_2200x2200x275nmBox')
RBDsPath = notebookPath.joinpath('blender_RBD_positions/RBD03_20x100nmFibrils_2200x2200x136nmBox')
savePath = notebookPath.joinpath('open3d_outputs')

In [ ]:
# Define simple functions
def to_s0to1(value, min=-180, max=180):
    """Adjust a linear scale from min to max to fit between 0 and 1"""
    shift = 0 - min
    max = max + shift
    return (value + shift) / max

def from_s0to1(value, min=-180, max=180):
    """Inverse of 'to_s0to1': Adjust a linear scale 0 to 1 to an arbitrary linear scale between min and max"""
    shift = 0 + min
    max = max - shift
    return (value * max) + shift

## Use open3D to build and visualize a point cloud

### Load RBD coords & XYZ euler rotations

In [ ]:
RBDs = sorted(RBDsPath.glob('*.txt'))
[f.name for f in RBDs]

In [ ]:
# RBD_coords = np.loadtxt(RBDs[-1]) * 10000 * 2  # convert um to Å, scale by factor of 2 in this case
RBD_coords = np.loadtxt(RBDs[-1]) * 10000  # convert um to Å
RBD_XYZrots = np.loadtxt(RBDs[0])

### Generate & save a fibril folder for each RBD 

In [ ]:
# Diameter distribution
targ_diam = 180  # diameter around which to pick a normal distribution in angstroms
sigma = 5  # standard deviation about diameter above

# Generate random distribution of diameters
rng = np.random.default_rng()
diams = rng.normal(loc=targ_diam, scale=sigma, size=RBD_coords.shape[0])
diams = np.round(diams, 2)

# Quick histogram check of distribution
plt.hist(diams, bins=100)
plt.show()

In [ ]:
# Generate fibril folders:
fibrilsPath = notebookPath.joinpath('custom_fibril_gen', f'RBD03_diam{targ_diam}sigma{sigma}_v1')
expPath = notebookPath.joinpath('custom_fibril_gen', 'PM6_CF_sol636_buf635_emp614.txt')

savepath = str(fibrilsPath)
exp_path = str(expPath)

num_fibrils = 1
length = 1000
flex = 0.01
e_dense = 0.0001  # 1 electron per 21.5x21.5x21.5 Å box -> 0.0001006
fuzz_length = 0
fuzz_density = 0
q_norm_idx = 9

flexes_dict = {0.05:'0p05', 0.01:'0p01',0.005:'0p005', 0.001:'0p001', 0.0001:'0p0001'}
dens_dict = {0.0001:'0p0001', 0.01:'0p01'}
# for i, diam in enumerate(tqdm(diams)):
#     for flex in flexes:
#         flex_label = flexes_dict[flex]
#         dens_label = dens_dict[e_dense]
#         test_fibril_scattering_par(
#             num_fibrils,length,diam,flex,e_dense,exp_path,
#             savepath + f'/dtotal{diam}_length{length}_flex{flex_label}_edens{dens_label}_idx{i}/'
#             )

for i, diam in enumerate(tqdm(diams)):
    flex_label = flexes_dict[flex]
    test_fibril_scattering_par(
        num_fibrils, length, diam, flex, e_dense, fuzz_density, fuzz_length, q_norm_idx, exp_path,
        savepath + f'/dtotal{diam}_length{length}_flex{flex_label}_idx{i}/'
        )

In [ ]:
# def sq_vec(pos,qs):
#     '''
#     Calculates the scattering profile using the debye equation. 

#     Input
#       pos = scatterer positions in 3D cartesian coordinates (nx3 array)
#       qs = list of q values to evaluate scattering intensity at
#     '''
#     ### initialize sq array for same length of q-range
#     nbins = len(qs)
#     sq = np.zeros((nbins))
#     ### number of atoms in fibril to be simulated
#     natoms = len(pos)
#     ### calculate contribution to scattering for each pair of atom
#     for i in tqdm(range(natoms)):
#         ### find displacements from current points to all subsequent points
#         ### prevents double counting of displacements
#         all_disp = pos[i,:]-pos[(i+1):,:]
        
#         ### calculate the distance from displacement vector ###
#         rij = np.sqrt(np.sum(np.square(all_disp),axis=1))

#         #create array of q vectors and radiuses
#         qi = qs[np.newaxis,:]
#         R = rij[:,np.newaxis]
#         ### add scattering intensity contribution for given pair of atoms
#         increment = np.sum(ne.evaluate("2*sin(R*qi)/(R*qi)"),axis=0)
#         print(increment.shape)
#         sq = sq+increment

#     return sq

In [ ]:
# pos = np.load(fibrilsPath.joinpath('dtotal188.63_length1000_flex0p01_idx1/flex_cylinder_num0.npy'))
# qs = np.loadtxt(expPath)[:,0]
# ints = sq_vec(pos, qs)
# ints

In [ ]:
# plt.loglog(qs, ints)
# plt.show()

### Build full point cloud with a single selected fibril

In [ ]:
fibrilPath = notebookPath.joinpath('fibrils_for_andrew/nofuzz_fibril_oct23/dtotal400_length2000_flex0p0001_edens0p0001')
fibril_npys = sorted(fibrilPath.glob('*.npy'))
[f.name for f in fibril_npys]

In [ ]:
# Add coordinates for each box, add to point cloud mesh immediately
pc = o3d.geometry.PointCloud()

# Load individual fibril (for now just placing this one everywhere... will be in loop when generating for each box)
backbone_axs = np.load(fibril_npys[0])
backbone_coords = np.load(fibril_npys[1])
backbone_css = np.load(fibril_npys[2])
fibril_points_arr = np.load(npys[3])[::5]

# Shift along z so that its midpoint is at the origin
zshift = np.zeros_like(fibril_points_arr)
zshift[:,2] = backbone_coords[:,2].max() / 2
fibril_points_arr = fibril_points_arr - zshift

# Load RBD coords & XYZ euler rotations, specify how many boxes to fill
RBD_coords = np.loadtxt(RBDs[-1]) * 10000 * 2 # convert um to Å
RBD_XYZrots = np.loadtxt(RBDs[0])
boxes_to_fill = -1  # -1 is all films
for RBD_XYZrot, RBD_coord in tqdm(zip(RBD_XYZrots[:boxes_to_fill], RBD_coords[:boxes_to_fill]), total=len(RBD_XYZrots[:boxes_to_fill])):
    # Apply euler rotation to each fibril and backbone axial pointers
    # Apply pre rotation correction if necessary
    r_pre_rotation = R.from_euler('y', 0, degrees=True)
    pre_rotated_fibril = r_pre_rotation.apply(fibril_points_arr)
    pre_rotated_bb_axs = r_pre_rotation.apply(backbone_axs)
    pre_rotated_bb_coords = r_pre_rotation.apply(backbone_coords)
    # Rotate corrected fibril orientation
    r_RBD = R.from_euler("xyz", RBD_XYZrot, degrees=True)
    rotated_fibril = r_RBD.apply(pre_rotated_fibril)
    rotated_backbone_axs = r_RBD.apply(pre_rotated_bb_axs)
    rotated_backbone_coords = r_RBD.apply(pre_rotated_bb_coords)

    # # Convert rotated backbone points to 2 Euler angles; PARALLEL to backbone (select backbone axial pointer)
    # phi = np.rad2deg(np.arctan2(rotated_backbone_axs[:,0], rotated_backbone_axs[:,1]))
    # theta = np.rad2deg(np.arccos(rotated_backbone_axs[:,2]))
    # euler_ZY_rot = np.vstack((phi,theta)).T
    # # Assign euler angle to each fibril point by its nearest backbone point:
    # fibril_ZYrots = np.empty((0,2))
    # for fibril_point in fibril_points_arr[:]:
    #     backbone_displacements = backbone_coords - fibril_point
    #     backbone_distances = np.sqrt(backbone_displacements[:,0]**2 + backbone_displacements[:,1]**2 + backbone_displacements[:,2]**2)
    #     backbone_minimum_index = (backbone_distances == backbone_distances.min()).nonzero()[0][0]
    
    #     fibril_ZYrots = np.append(fibril_ZYrots, euler_ZY_rot[backbone_minimum_index].reshape(1,2), axis=0)

    # Convert rotated backbone points to 2 Euler angles; PERPENDICULAR to backbone (extract euler angle for shortest pointer to backbone)
    fibril_ZYrots = np.empty((0,2))
    for fibril_point in rotated_fibril[:]:
        backbone_displacements = rotated_backbone_coords - fibril_point
        backbone_distances = np.sqrt(backbone_displacements[:,0]**2 + backbone_displacements[:,1]**2 + backbone_displacements[:,2]**2)
        backbone_minimum_index = (backbone_distances == backbone_distances.min()).nonzero()[0][0]
    
        shortest_displacement = backbone_displacements[backbone_minimum_index]
        shortest_displacement = shortest_displacement / shortest_displacement.max()  # make a unit vector
        phi = np.rad2deg(np.arctan2(shortest_displacement[0], shortest_displacement[1]))
        theta = np.rad2deg(np.arccos(shortest_displacement[2]))
        euler_ZY_rot = np.vstack((phi,theta)).T
    
        fibril_ZYrots = np.append(fibril_ZYrots, euler_ZY_rot, axis=0)
    
    # Encode Euler angles into RGB values (set B=1, represent S)`
    fibril_RGB_values = np.array([to_s0to1(fibril_ZYrots[:,0], min=-180, max=180),  # psi
                                  to_s0to1(fibril_ZYrots[:,1], min=0, max=180),  # theta
                                  np.ones((fibril_ZYrots[:,0].shape))]).T  # S (ones)
    
    # Move fibril coordinates to RBD location
    moved_fibril = (rotated_fibril + RBD_coord) / 2  # divide by 2 just for now to undo scaling from before
    # moved_fibril = (rotated_fibril + RBD_coord)  
    # moved_fibril = (fibril_points_arr + RBD_coord)  
    
    # Add points and colors to point cloud
    pc.points.extend(moved_fibril)
    pc.colors.extend(fibril_RGB_values)

In [ ]:
pc

In [ ]:
o3d.visualization.EV.set(pc)

In [ ]:
# o3d.io.write_point_cloud('testing.ply', pc)

## Use open3D to voxelize point cloud

In [ ]:
# np.asarray(pc.points).max() / 1024

In [ ]:
1024*1024*128

In [ ]:
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pc, voxel_size=21.5)
voxel_grid

In [ ]:
# visualize, will need to restart kernel after :O
o3d.visualization.draw_geometries([voxel_grid])

### Save open3d voxel grid as binary .ply file

In [ ]:
savePath

In [ ]:
# o3d.io.write_voxel_grid(str(savePath.joinpath('RBD01_perp2bb_voxel_grid_v2.ply')), voxel_grid)
o3d.io.write_voxel_grid(str(savePath.joinpath('RBD03_perp2bb_voxel_grid_v1.ply')), voxel_grid)
# o3d.io.write_voxel_grid(str(savePath.joinpath('RBD02_para2bb_voxel_grid_v1.ply')), voxel_grid)
# o3d.io.write_voxel_grid(str(savePath.joinpath('RBD02_perp2bb_voxel_grid_v1.ply')), voxel_grid)

In [ ]:
# Load saved file & check contents

loaded_voxel_grid = o3d.io.read_voxel_grid(str(savePath.joinpath('RBD03_perp2bb_voxel_grid_v1.ply')))
loaded_voxel_grid

In [ ]:
# Load saved file & check contents

loaded_voxel_grid = o3d.io.read_voxel_grid(str(savePath.joinpath('RBD03_para2bb_voxel_grid_v1.ply')))
loaded_voxel_grid

In [ ]:
# Convert voxel grid to list of voxels, with grid index & color

voxels = voxel_grid.get_voxels()  # returns list of voxels
# indices = np.stack(list(vx.grid_index for vx in voxels))
# colors = np.stack(list(vx.color for vx in voxels))

voxels